In [1]:
import os
import math
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
cwd = os.getcwd()
%load_ext autoreload
%autoreload 2

In [2]:
import os
cwd = os.getcwd()
os.chdir(os.path.join(cwd,'../../..'))
import ExMAS.main
import ExMAS.utils
from ExMAS.utils import inData as inData
from ExMAS.main import matching
from ExMAS.game import games, pricings, prunings, pipeline

In [3]:
params = ExMAS.utils.get_config('ExMAS/data/configs/default.json', root_path=None)  # load the default
params.t0 = pd.to_datetime(params.t0)
params.logger_level = 'INFO'
params.matching_obj = 'u_veh'

# parameterization
params.veh_cost = 2.3 * params.VoT / params.avg_speed  # operating costs per kilometer
params.fixed_ride_cost = 1  # ride fixed costs (per vehicle)
params.time_cost = params.VoT  # travellers' cost per travel time
params.wait_cost = params.time_cost * 1.5  # and waiting
params.sharing_penalty_fixed = 0  # fixed penalty (EUR) per
params.sharing_penalty_multiplier = 0  # fixed penalty (EUR) per

params.max_detour = 120  # windows
params.max_delay = 120  # windows

from ExMAS.utils import inData as inData

inData = ExMAS.utils.load_G(inData, params, stats=True)  # download the graph

# initial settings
params.minmax = 'min'
params.multi_platform_matching = False
params.assign_ride_platforms = True
params.nP = 100
params.simTime = 0.15
params.shared_discount = 0.2

# prepare ExMAS
inData = ExMAS.utils.generate_demand(inData, params)  # generate requests


PRUNINGS = dict()  # algorithms to apply and their names
PRUNINGS['EXMAS'] = prunings.algo_EXMAS
PRUNINGS['TNE'] = prunings.algo_TNE
PRUNINGS['HERMETIC'] = prunings.algo_HERMETIC
PRUNINGS['RUE'] = prunings.algo_RUE
PRUNINGS['RSIE'] = prunings.algo_RSIE
PRUNINGS['TSE'] = prunings.algo_TSE

PRICINGS = dict()  # pricings to apply and their names
PRICINGS['UNIFORM'] = pricings.uniform_split
PRICINGS['EXTERNALITY'] = pricings.externality_split
PRICINGS['RESIDUAL'] = pricings.residual_split
PRICINGS['SUBGROUP'] = pricings.subgroup_split

# clear
inData.sblts.mutually_exclusives = []
inData.sblts.rides['pruned'] = True
inData = ExMAS.main(inData, params, plot=False)  # create feasible groups

inData = games.prepare_PoA(inData)  # prepare data structures
inData = pricings.update_costs(inData, params)  # determine costs per group and per traveller
for PRICING, pricing in PRICINGS.items():
    inData = pricing(inData)  # apply pricing strategy

inData.results.rides = inData.sblts.rides.copy()  # copy tables to collect results
inData.results.rm = inData.sblts.rides_multi_index.copy()
inData.results.KPIs = dict()

for PRICING, pricing in PRICINGS.items():
    print(PRICING)
    inData = pricing(inData)  # apply pricing strategy
    for PRUNING, pruning in PRUNINGS.items():
        print(PRUNING)
        inData = pruning(inData, price_column=PRICING)  # apply pruning strategies for a given pricing strategy
    for PRUNING, pruning in PRUNINGS.items():  # perform assignment for single prunings
        print(PRUNING)
        inData = pipeline.single_eval(inData, params,
                             EXPERIMENT_NAME='jupyter',
                             MATCHING_OBJS=['total_group_cost'],  # this can be more
                             PRUNINGS=[PRUNING],  # and this can be more
                             PRICING=PRICING,  # this is taken from first level loop
                             minmax=('min', 'max'))  # direction BPoA, WPoA

12-05-22 15:59:06-INFO-Initializing pairwise trip shareability between 100 and 100 trips.
12-05-22 15:59:06-INFO-creating combinations
12-05-22 15:59:06-INFO-9900	 nR*(nR-1)
12-05-22 15:59:06-INFO-Reduction of feasible pairs by 99.67%
12-05-22 15:59:06-INFO-Degree 2 	Completed
12-05-22 15:59:06-INFO-trips to extend at degree 2 : 79
12-05-22 15:59:06-INFO-At degree 2 feasible extensions found out of 4 searched
12-05-22 15:59:06-INFO-Degree 3 	Completed
12-05-22 15:59:06-INFO-trips to extend at degree 3 : 4
12-05-22 15:59:06-INFO-At degree 3 feasible extensions found out of 0 searched
12-05-22 15:59:06-INFO-Degree 4 	Completed
12-05-22 15:59:06-INFO-No more trips to exted at degree 4
12-05-22 15:59:06-INFO-Matching 100 trips to 183 rides in order to min u_veh
12-05-22 15:59:06-INFO-Problem solution: Optimal. 
Total costs for single trips:         32,669 
reduced by matching to:               27,413
12-05-22 15:59:06-INFO-Calculations  completed
12-05-22 15:59:06-INFO-DotMap(VehHourTrav=2

---
(c) Rafał Kucharski, Delft, 2021